# Mission to Mars

![mission_to_mars](Images/mission_to_mars.jpg)

In this assignment, you will build a web application that scrapes various websites for data related to the Mission to Mars and displays the information in a single HTML page. The following outlines what you need to do.

## Step 1 - Scraping

Complete your initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.

* Create a Jupyter Notebook file called `mission_to_mars.ipynb` and use this to complete all of your scraping and analysis tasks. The following outlines what you need to scrape.

In [1]:
# Complete

### NASA Mars News

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

```python
# Example:
news_title = "NASA's Next Mars Mission to Investigate Interior of Red Planet"

news_p = "Preparation of NASA's next spacecraft to Mars, InSight, has ramped up this summer, on course for launch next May from Vandenberg Air Force Base in central California -- the first interplanetary launch in history from America's West Coast."
```

In [2]:
# imports and froms
import datetime
import json
import pandas as pd
import pymongo

import re
import time
from bs4 import BeautifulSoup
from splinter import Browser

# define variables
executable_path = {'executable_path': 'c:\windows\chromedriver\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)
url_news = 'https://mars.nasa.gov/news/'

# create objects
browser.visit(url_news)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
articles = soup.find('div', class_="grid_layout")

# scrape 
# time.sleep(10)
news_title = articles.find('h3').text
news_p = articles.find('div', class_='article_teaser_body').text
print(news_title)
print(news_p)

NASA's InSight Detects First Likely 'Quake' on Mars
While their causes are still unknown, one of three shaking events looks a lot like the quakes detected on the Moon by the Apollo missions.


### JPL Mars Space Images - Featured Image

* Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

* Make sure to find the image url to the full size `.jpg` image.

* Make sure to save a complete url string for this image.

```python
# Example:
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'
```

In [3]:
# load url into soup
url_img = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url_img)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
articles = soup.find_all('article', class_='carousel_item')

# search through soup and save the url
for article in articles:
    featured_image_url = 'https://www.jpl.nasa.gov' + str(re.findall(r"'(.*?)'", article['style'])[0])
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19347-1920x1200.jpg


### Mars Weather

* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.

```python
# Example:
mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'
```

In [4]:
# load url into soup
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# load into soup and get the first tweet
article = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")
mars_weather = article.text.split('pic.twitter')[0]
mars_weather

'Mars, I hear you. I’ve detected some quiet but distinct shaking on #Mars. The faint rumbles appear to have come from the inside of the planet, and are still being studied by my team. Take a listen.http://go.nasa.gov/2GCEBtm\xa0'

### Mars Facts

* Visit the Mars Facts webpage [here](http://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

* Use Pandas to convert the data to a HTML table string.

In [5]:
# load url into soup
url = 'http://space-facts.com/mars/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# save the table to a dataframe
data = []
table = soup.find('table')
rows = table.find_all('tr')

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) 
data_df = pd.DataFrame(data, columns=['key', 'value'])

# save the dataframe to html
data_df.to_html('templates/mars_facts.html', header=False, index=False)

### Mars Hemispheres

* Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

```python
# Example:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
]
```

In [6]:
# load url into soup
url_img = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_img)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# search through soup and save image dictionaries to a list
articles = soup.find_all('a', class_='itemLink product-item')
hemi_list = []
count = 0
for article in articles:
    Href_partial = article['href'].replace('/search/map', '/download')
    if count%2 != 0:
        title = article.text.replace('Hemisphere Enhanced', '')
        img_url=(f'https://astropedia.astrogeology.usgs.gov{Href_partial}.tif/full.jpg')
        d = dict(title=title, img_url=img_url)
        hemi_list.append(d)
    count += 1
hemi_list

[{'title': 'Cerberus ',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli ',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major ',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris ',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [7]:
# create a dictionary for loading into mongodb
mars_data = {
    'news_title':news_title,
    'news_p':news_p,
    'featured_image_url':featured_image_url,
    'mars_weather':mars_weather,
    'hemi_list':hemi_list
}

In [8]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.mars_app

# Declare the collection
collection = db.mars_app

# Insert the document into the database
# The database and collection, if they don't already exist, will be created at this point.
collection.insert_one(mars_data)

- - -

## Step 2 - MongoDB and Flask Application

Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above.

* Start by converting your Jupyter notebook into a Python script called `scrape_mars.py` with a function called `scrape` that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.

* Next, create a route called `/scrape` that will import your `scrape_mars.py` script and call your `scrape` function.

  * Store the return value in Mongo as a Python dictionary.

* Create a root route `/` that will query your Mongo database and pass the mars data into an HTML template to display the data.

* Create a template HTML file called `index.html` that will take the mars data dictionary and display all of the data in the appropriate HTML elements. Use the following as a guide for what the final product should look like, but feel free to create your own design.

![final_app_part1.png](Images/final_app_part1.png)
![final_app_part2.png](Images/final_app_part2.png)

- - -

## Step 3 - Submission

To submit your work to BootCampSpot, create a new GitHub repository and upload the following:

1. The Jupyter Notebook containing the scraping code used.

2. Screenshots of your final application.

3. Submit the link to your new repository to BootCampSpot.

https://github.com/gilbertduenas/Mars-Reporting

## Hints

* Use Splinter to navigate the sites when needed and BeautifulSoup to help find and parse out the necessary data.

* Use Pymongo for CRUD applications for your database. For this homework, you can simply overwrite the existing document each time the `/scrape` url is visited and new data is obtained.

* Use Bootstrap to structure your HTML template.

## Copyright

© 2019 Trilogy Education Services. All Rights Reserved.